In [1]:
from langgraph.graph import StateGraph, START, END

from typing_extensions import TypedDict
import operator

from typing import List, TypedDict, Annotated



/Users/camron/Documents/Dev/promptnado/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [1]:
%pip install -qU -e .

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from promptnado import Promptnado, Example, LangsmithDataset, Rule
from langchain.schema.messages import AIMessage
from pydantic.v1 import BaseModel, Field


example_system_prompt = """You are a helpful assistant. 

Rules:
- You are only allowed to talk about coding
- <HERE>
- Try to be concise"""

example_instruction = "The agent should only respond in English."

examples = [[AIMessage(content="Hola, como estas?")], "Hi there!", Example(
    input="Hi there!", reference_output="Hey!")]


In [ ]:
# dataset = LangsmithDataset(dataset_name="Promptnado_476666")

pn = Promptnado(example_system_prompt, example_instruction,
                examples, max_attempts=2, experiment_name="graph")

In [ ]:
class State(TypedDict):
    # List of rules that have not been tested yet
    untested_rules: List[Rule]
    tested_rules: List[Rule]           # List of rules that have been tested
    current_rule: Rule                 # The rule currently being tested
    current_prompt: str               # The prompt associated with the current rule
    solved: bool                   # Indicates whether the current rule has solved the problem
    attempt: int               # The number of attempts made with the current rule
    successful_prompt: str  # The prompt that was successful, if any
    # test_results: List[str]           # Results of each test performed



In [2]:
# Nodes
def generate_rules(state: State):
    pn._generate_rules()
    return {"untested_rules": pn.rules}


def next_rule(state: State):
    """Set the state to be ready to test the next rule"""
    pn._next_rule()
    return {"current_rule": pn.current_rule, "current_prompt": pn.current_prompt, "untested_rules": pn.rules}


def test_rule(state: State):
    # Pop rule from state["rules"]
    current_rule = state["current_rule"]
    results = pn._test_rule(current_rule)
    solved = pn._is_solved(results)
    return {"solved": solved, "attempt": state["attempt"] + 1, "successful_prompt": pn.current_prompt}


def success(state: State):
    print("\n\nSolved!! Current prompt can be found at `self.successful_prompt`\n\n")
    # ANSI escape codes for green text and reset
    GREEN = "\033[92m"
    RESET = "\033[0m"

    green_rule = f"{GREEN}{pn.current_rule.prompt}{RESET}"
    highlighted_prompt = pn.system_prompt.replace(
        pn.rule_token, green_rule)

    print(
        f"Successful prompt:\n====================\n{highlighted_prompt}\n=================")
    print(pn.dataset.dataset.url)
    return {"current_rule": pn.current_rule, "current_prompt": pn.current_prompt, "solved": True, "attempts": pn.attempts}

def failure(state: State):
    return {"tested_rules": pn.tested_rules, "solved": False, "attempts": pn.attempts}

NameError: name 'State' is not defined

In [ ]:
# edges
def route(state: State):
    if pn.solved:
        return "success"
    
    # If weve reached the max attempts route to failure
    if pn.attempts >= pn.max_attempts:
        return "failure"
    
    # If we have no more rules left, generate more
    if not pn.rules:
        return "generate_rules"
    
    return "next"

In [ ]:
def create_graph():
    graph = StateGraph(State)

    # Add nodes
    graph.add_node("GenerateRules", generate_rules)
    graph.add_node("NextRule", next_rule)
    graph.add_node("TestRule", test_rule)
    graph.add_node("Success", success)
    graph.add_node("Failure", failure)

    # Add edges
    graph.add_edge("GenerateRules", "NextRule")
    graph.add_edge("NextRule", "TestRule")
    
    # Conditional edges from test_rule
    graph.add_conditional_edges(
        "TestRule",
        route,
        {
            "success": "Success",
            "generate_rules": "GenerateRules",
            "next": "NextRule",
            "failure": "Failure"
        }
    )

    # Set the entry point
    graph.set_entry_point("GenerateRules")
    app = graph.compile()
    return app

app = create_graph()
